## Black Box Approach for all 10 genres classification

We aim at training a Neural Network to distinguish between all the different genres

In [ ]:
#importing google drive
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
#setting the working directory
%cd /gdrive/MyDrive/polimi/NAML/NAML_proj/

/gdrive/MyDrive/polimi/NAML/NAML_proj


In [ ]:
import tensorflow as tf
from PIL import Image
import numpy as np
import os
import random
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
import librosa
import os
import copy

tfk = tf.keras
tfkl = tf.keras.layers
seed = 42

The dataset is composed of Mel-Spectrograms of each audio sample, labeled with respect to each genre

In [ ]:
dataset = []
genres = {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 
        'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}
n_genres = 10

for genre, genre_number in genres.items():
    for filename in os.listdir(f'dataset_old/genres/{genre}'):
        songname = f'dataset_old/genres/{genre}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=29.7)
        ps = librosa.feature.melspectrogram(y=y, sr=sr, hop_length = 256, n_fft = 512)
        ps = librosa.power_to_db(ps**2)
        dataset.append( (ps, genre_number) )
    print(str(genre+' done'))

blues done
classical done
country done
disco done
hiphop done
jazz done
metal done
pop done
reggae done
rock done


We split the dataset according to the following composition:


*   70% training set
*   20% validation set
*   10% test set

Maintaining equal proportions amongst classes



In [ ]:
order = np.arange(start = 0, stop = 100, step = 1)

np.random.seed(seed)

training_validation = []
test = []

for i in range(n_genres):
  shuffle = np.random.permutation(order)
  for k in range(90):
    training_validation.append(dataset[i*100 + shuffle[k]])
  for m in range(10):
    test.append(dataset[i*100 + shuffle[m+90]])

In [ ]:
X_tr_val, Y_tr_val = zip(*training_validation)
X_test, Y_test = zip(*test)

X_tr_val = np.array([x.reshape( (128, 2559, 1) ) for x in X_tr_val])
X_test = np.array([x.reshape( (128, 2559, 1) ) for x in X_test])


Y_tr_val = np.array(tfk.utils.to_categorical(Y_tr_val, n_genres))
Y_test = np.array(tfk.utils.to_categorical(Y_test, n_genres))

The proposed model is composed as a stack of convolutional layer, followed by a Global Average Pooling layer leading to a fully connected section.

Optimization is performed as a Batch version of Adam optimizer algorithm

In [ ]:
def build_model(input_shape, n_units):
# Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    conv1 = tfkl.Conv2D(
        name = 'conv_1',
        filters=4,
        kernel_size=(5, 5),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(input_layer)
    conv1_2 = tfkl.Conv2D(
        name = 'conv_1_2',
        filters=8,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(conv1)
    pool1 = tfkl.MaxPooling2D(
        name = 'pool_1',
        pool_size = (2, 2)
    )(conv1_2)

    conv2 = tfkl.Conv2D(
        name = 'conv_2',
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool1)
    conv2_2 = tfkl.Conv2D(
        name = 'conv_2_2',
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(conv2)
    pool2 = tfkl.MaxPooling2D(
        name = 'pool_2',
        pool_size = (2, 2)
    )(conv2_2)

    conv3 = tfkl.Conv2D(
        name = 'conv_3',
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool2)
    pool3 = tfkl.MaxPooling2D(
        name = 'pool_3',
        pool_size = (2, 2)
    )(conv3)

    conv4 = tfkl.Conv2D(
        name = 'conv_4',
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool3)
    pool4 = tfkl.MaxPooling2D(
        name = 'pool_4',
        pool_size = (2, 2)
    )(conv4)

    conv5 = tfkl.Conv2D(
        name = 'conv_5',
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool4)
    pool5 = tfkl.MaxPooling2D(
        name = 'pool_5',
        pool_size = (2, 2)
    )(conv5)

    conv6 = tfkl.Conv2D(
        name = 'conv_6',
        filters=286,
        kernel_size=(1, 1),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool5)
    
    global_average = tfkl.GlobalAveragePooling2D(name = 'GAP')(conv6)
    global_average = tfkl.Dropout(0.3, seed=seed, name = 'Dropout_GAP')(global_average)
    
    classifier_layer = tfkl.Dense(units=64, name='Classifier', activation='relu')(global_average)
    
    classifier_layer = tfkl.Dropout(0.3, seed=seed, name = 'Dropout_Classifier')(classifier_layer)
    classifier_layer_2 = tfkl.Dense(units=32, name='Classifier_2', activation='relu')(classifier_layer)
    classifier_layer_2 = tfkl.Dropout(0.3, seed=seed, name = 'Dropout_Classifier_2')(classifier_layer_2)
    output_layer = tfkl.Dense(units=n_units, activation='softmax', name='Output')(classifier_layer_2)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model

    return model

In [8]:
metrics_accuracy = []
metrics_precision = []
metrics_recall = []
metrics_f1 = []

n_trials = 10
input_shape = (128, 2559, 1)
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)
adaptive_LR = tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-4)

order = np.arange(start = 0, stop = 90, step = 1)
np.random.seed(seed)
#initialization
X_train = copy.deepcopy(X_tr_val[0:700])
Y_train = copy.deepcopy(Y_tr_val[0:700])
X_valid = copy.deepcopy(X_tr_val[0:200])
Y_valid = copy.deepcopy(Y_tr_val[0:200])


for i in range(n_trials):
  model = build_model(input_shape, n_genres)
  
  #random split in training and validation
  
  for j in range(n_genres):
    shuffle = np.random.permutation(order)
    for k in range(70):
      X_train[k] = copy.deepcopy(X_tr_val[i*90 + shuffle[k]])
      Y_train[k] = copy.deepcopy(Y_tr_val[i*90 + shuffle[k]])
    for m in range(20):
      X_valid[m] = copy.deepcopy(X_tr_val[i*90 + shuffle[m+70]])
      Y_valid[m] = copy.deepcopy(Y_tr_val[i*90 + shuffle[m+70]])

  standard_history = model.fit(
    x = X_train,
    y = Y_train,
    epochs = 500,
    batch_size = 100,
    validation_data= (X_valid, Y_valid),
    callbacks = [early_stopping, adaptive_LR]
    )
  predictions = model.predict(X_test)
  metrics_accuracy.append(accuracy_score(np.argmax(Y_test, axis=-1), np.argmax(predictions, axis=-1)))
  metrics_precision.append(precision_score(np.argmax(Y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro'))
  metrics_recall.append(recall_score(np.argmax(Y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro'))
  metrics_f1.append(f1_score(np.argmax(Y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro'))
  model.save(str('/gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_1_'+str(i)))
  print(metrics_accuracy)
  

Epoch 1/500
7/7 [==============================] - 29s 2s/step - loss: 2.2792 - accuracy: 0.1200 - val_loss: 2.2118 - val_accuracy: 0.1000 - lr: 0.0010
Epoch 2/500
7/7 [==============================] - 10s 1s/step - loss: 2.2522 - accuracy: 0.1029 - val_loss: 2.2038 - val_accuracy: 0.1000 - lr: 0.0010
Epoch 3/500
7/7 [==============================] - 10s 1s/step - loss: 2.1953 - accuracy: 0.1171 - val_loss: 2.1123 - val_accuracy: 0.3250 - lr: 0.0010
Epoch 4/500
7/7 [==============================] - 10s 1s/step - loss: 2.1756 - accuracy: 0.1257 - val_loss: 2.1427 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 5/500
7/7 [==============================] - 10s 1s/step - loss: 2.1601 - accuracy: 0.1400 - val_loss: 2.1278 - val_accuracy: 0.0050 - lr: 0.0010
Epoch 6/500
7/7 [==============================] - 10s 1s/step - loss: 2.1283 - accuracy: 0.1814 - val_loss: 1.7473 - val_accuracy: 0.4150 - lr: 0.0010
Epoch 7/500
7/7 [==============================] - 10s 1s/step - loss: 2.0940 - accu

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_1_0/assets


INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_1_0/assets


[0.64]
Epoch 1/500
7/7 [==============================] - 12s 2s/step - loss: 2.3354 - accuracy: 0.1486 - val_loss: 2.1574 - val_accuracy: 0.5500 - lr: 0.0010
Epoch 2/500
7/7 [==============================] - 10s 1s/step - loss: 2.1856 - accuracy: 0.1700 - val_loss: 2.1260 - val_accuracy: 0.5500 - lr: 0.0010
Epoch 3/500
7/7 [==============================] - 10s 1s/step - loss: 2.1535 - accuracy: 0.1857 - val_loss: 2.1861 - val_accuracy: 0.5300 - lr: 0.0010
Epoch 4/500
7/7 [==============================] - 10s 1s/step - loss: 2.1266 - accuracy: 0.1857 - val_loss: 2.0305 - val_accuracy: 0.5500 - lr: 0.0010
Epoch 5/500
7/7 [==============================] - 10s 1s/step - loss: 1.9777 - accuracy: 0.2614 - val_loss: 1.8794 - val_accuracy: 0.5400 - lr: 0.0010
Epoch 6/500
7/7 [==============================] - 10s 2s/step - loss: 1.9200 - accuracy: 0.2886 - val_loss: 1.8817 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 7/500
7/7 [==============================] - 10s 1s/step - loss: 1.8770 - a

...

[0.64, 0.36, 0.52, 0.52, 0.53, 0.58, 0.57, 0.59, 0.66]
Epoch 1/500
7/7 [==============================] - 12s 2s/step - loss: 2.2305 - accuracy: 0.1486 - val_loss: 2.4079 - val_accuracy: 0.4550 - lr: 0.0010
Epoch 2/500
7/7 [==============================] - 10s 1s/step - loss: 2.2123 - accuracy: 0.1371 - val_loss: 2.2214 - val_accuracy: 0.4700 - lr: 0.0010
Epoch 3/500
7/7 [==============================] - 10s 1s/step - loss: 2.1536 - accuracy: 0.2086 - val_loss: 2.0794 - val_accuracy: 0.4400 - lr: 0.0010
Epoch 4/500
7/7 [==============================] - 10s 1s/step - loss: 2.0834 - accuracy: 0.2100 - val_loss: 2.0568 - val_accuracy: 0.4250 - lr: 0.0010
Epoch 5/500
7/7 [==============================] - 10s 1s/step - loss: 2.0381 - accuracy: 0.2171 - val_loss: 2.1108 - val_accuracy: 0.4100 - lr: 0.0010
Epoch 6/500
7/7 [==============================] - 10s 1s/step - loss: 1.9780 - accuracy: 0.2329 - val_loss: 1.9694 - val_accuracy: 0.4550 - lr: 0.0010
Epoch 7/500
7/7 [================

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_1_9/assets


INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_1_9/assets


[0.64, 0.36, 0.52, 0.52, 0.53, 0.58, 0.57, 0.59, 0.66, 0.62]


In [9]:
metrics_accuracy = np.array(metrics_accuracy)
metrics_precision = np.array(metrics_precision)
metrics_recall = np.array(metrics_recall)
metrics_f1 = np.array(metrics_f1)

print(str('mean accuracy = ' + str(metrics_accuracy.mean() )))
print(str('mean precision = ' + str(metrics_precision.mean() )))
print(str('mean recall = ' + str(metrics_recall.mean() )))
print(str('mean f1 = ' + str(metrics_f1.mean() )))

print('***___***')

print(str('accuracy standard deviation = ' + str(metrics_accuracy.std() )))
print(str('precision standard deviation = ' + str(metrics_precision.std() )))
print(str('recall standard deviation = ' + str(metrics_recall.std() )))
print(str('f1 standard deviation = ' + str(metrics_f1.std() )))

print('***___***')

print(str('accuracy range = ' + str(metrics_accuracy.max() - metrics_accuracy.min() )))
print(str('precision range = ' + str(metrics_precision.max() - metrics_precision.min() )))
print(str('recall range = ' + str(metrics_recall.max() - metrics_recall.min() )))
print(str('f1 range = ' + str(metrics_f1.max() - metrics_f1.min() )))

print('***___***')

print(str('min accuracy = ' + str(metrics_accuracy.min() )))
print(str('min precision = ' + str(metrics_precision.min() )))
print(str('min recall = ' + str(metrics_recall.min() )))
print(str('min f1 = ' + str(metrics_f1.min() )))


mean accuracy = 0.5589999999999999
mean precision = 0.4589567930592463
mean recall = 0.559
mean f1 = 0.4809294053523373
***___***
accuracy standard deviation = 0.08092589202474076
precision standard deviation = 0.09634691114087213
recall standard deviation = 0.08092589202474075
f1 standard deviation = 0.08453205433937733
***___***
accuracy range = 0.30000000000000004
precision range = 0.33913396544975494
recall range = 0.30000000000000004
f1 range = 0.3112156017461588
***___***
min accuracy = 0.36
min precision = 0.2493031529873635
min recall = 0.36
min f1 = 0.28497037665377845


In [10]:
print(metrics_accuracy)
print(metrics_precision)
print(metrics_recall)
print(metrics_f1)

[0.64 0.36 0.52 0.52 0.53 0.58 0.57 0.59 0.66 0.62]
[0.52626263 0.24930315 0.41777778 0.38577259 0.45573667 0.44083585
 0.41738562 0.54021368 0.56784285 0.58843712]
[0.64 0.36 0.52 0.52 0.53 0.58 0.57 0.59 0.66 0.62]
[0.57108786 0.28497038 0.44411483 0.43873082 0.4423673  0.4920717
 0.47062096 0.51181743 0.59618598 0.5573268 ]
